In [1]:
with open('input.txt') as f:
    lines = f.readlines()

In [2]:
from functools import cache

In [3]:
def is_valid(s):
    if '@#' in s or '#@' in s:
        return False
    else:
        return True

In [4]:
def place_substring(s, length):
    pos = []
    for i in range(len(s) - length+1):
        str_overwritten = s[i:i+length]
        if '.' in str_overwritten or '@' in str_overwritten:
            continue
        # Check for # before
        for j in range(i-1):
            if s[j] == '#':
                return tuple(pos) # No need to check further
        cur_str = s[:i] + length * '@' + s[i+length:]
        for j in range(i):
            if cur_str[j] == '?':
                cur_str = cur_str[:j] + '.' + cur_str[j+1:]
        
        if is_valid(cur_str):
            # Replace the chars with '.'
            cur_str = cur_str.replace('@?', '@.').replace('?@', '.@')
            pos.append(cur_str)
    return tuple(pos)

In [10]:
def get_all_pos(s, lengths, total_l):
    if s.count('#') == 0 and s.count('@') == total_l:
        return 1
    if len(lengths) == 0:
        return 0
    else:
        lengths_copy = lengths
        next_poss = place_substring(s, lengths_copy[0])
        lengths_copy = lengths_copy[1:]
        total = 0
        for pos in next_poss:
            # Check first if it could even be a solution:
            if pos.count('#') + pos.count('@') > total_l:
                continue
            elif pos.count('#') + pos.count('?') < sum(lengths_copy):
                continue
            else:
                print(pos)
                total += get_all_pos(pos, lengths_copy, total_l)
        return total
    

In [11]:
examples = []
for line in lines:
    example = line.strip().split(' ')
    #example[0] = "?".join([example[0] for _ in range(5)])
    #example[1] = ",".join([example[1] for _ in range(5)])             
    example[1] = [int(x) for x in example[1].split(',')]
    examples.append(example)

In [12]:
def process_example(example):
    return get_all_pos(example[0], tuple(example[1]), sum(example[1]))

In [14]:
examples[0]

['#?#???????#?.?', [3, 1, 2, 2]]

In [13]:
process_example(examples[0])

@@@.??????#?.?
@@@.@.????#?.?
@@@.@.@@.?#?.?
@@@.@.@@.@@..?
@@@.@.@@..@@.?
@@@.@..@@.#?.?
@@@.@..@@.@@.?
@@@..@.???#?.?
@@@..@.@@.#?.?
@@@..@.@@.@@.?
@@@...@.??#?.?
@@@....@.?#?.?


4

In [401]:
from tqdm import tqdm
from multiprocessing import Pool
import os



# Assuming 'examples' is a list of tuples
with Pool(os.cpu_count()) as pool:
    results = list(tqdm(pool.imap(process_example, examples), total=len(examples)))

total = sum(results)


  1%|▏         | 14/1000 [23:33<27:39:21, 100.98s/it]


KeyboardInterrupt: 

In [ ]:
len(results)

1000

In [ ]:
print(total)

7007

In [ ]:
7007, 322723, 46660346

In [15]:
from functools import cache


def solve_day(my_file):
  data = parse_data(my_file)
  print('Part 1: ', part1(data))
  print('Part 2: ', part2(data))


def parse_data(my_file):
  with open(my_file) as f:
    result = []
    for line in f.readlines():
      row, nums = line.split()
      nums = tuple(int(num) for num in nums.split(','))
      result.append((row, nums))
    return result


@cache
def springs_finder(row, nums):
  next_part = nums[1:]
  springs = (f"{spr*'.'}{'#'*nums[0]}."
             for spr in range(len(row) - sum(nums) - len(next_part)))
  valid = (len(spr) for spr in springs
           if all(r in (c, '?') for r, c in zip(row, spr)))
  return sum(springs_finder(row[v:], next_part)
             for v in valid) if next_part else sum('#' not in row[v:]
                                                   for v in valid)


def part1(data):
  return sum(springs_finder(r + '.', n) for r, n in data)


def part2(data):
  return part1((('?'.join([r] * 5), n * 5) for r, n in data))

solve_day('input.txt')

Part 1:  7007
Part 2:  3476169006222
